# Linking GLAM with EXIOBASE 3

This tutorial covers the linking of the [Global Guidance for Life Cycle Impact Assessment Indicators and Methods (GLAM)](https://www.lifecycleinitiative.org/activities/life-cycle-assessment-data-and-methods/global-guidance-for-life-cycle-impact-assessment-indicators-and-methods-glam/) with the Environmentally-Extended Multi-Regional Input Output database [EXIOBASE 3](https://onlinelibrary.wiley.com/doi/full/10.1111/jiec.12715).

The tutorial was tested with the latest version of both datasets:

- [GLAM V1.0.2024.10](https://www.lifecycleinitiative.org/activities/life-cycle-assessment-data-and-methods/global-guidance-for-life-cycle-impact-assessment-indicators-and-methods-glam/)
- [EXIOBASE 3.8.2](https://doi.org/10.5281/zenodo.5589597)

After the initial setup and data retrieval, the linking approach follows a two step approach. First, we translate the EXIOBASE stressor names to GLAM flow names; second, we characterise the flows with the GLAM characterization factors.
TODO: insert links to the headers later
The whole tutorial is self contained and automatically downloads all required data. The only pre-requisite is a [working installation of the latest version of Pymrio](https://pymrio.readthedocs.io/en/latest/installation.html). 

## Setup, folder definitions and data gathering

Here we import the required Python modules and define the folders to store the data. 

In [20]:
from pathlib import Path
import pymrio

import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Next, we specify were data should be stored

In [2]:
DATA_ROOT = Path("/tmp/glam_exio_tutorial") # set this to your data directory

EXIOBASE_STORAGE_FOLDER = DATA_ROOT / "exiobase"
GLAM_STORAGE_FOLDER = DATA_ROOT / "glam"

EXIOBASE_STORAGE_FOLDER.mkdir(parents=True, exist_ok=True)
GLAM_STORAGE_FOLDER.mkdir(parents=True, exist_ok=True)

Then, we download the data needed for the linking.
Here, we use EXIOBASE 3.8.2 in the product by product (pxp) classification for the year 2018.

In [3]:
pymrio.download_exiobase3(storage_folder=EXIOBASE_STORAGE_FOLDER, years=[2018], system="pxp", overwrite_existing=False)

Description: Download log of EXIOBASE3
MRIO Name: EXIO3
System: pxp
Version: 10.5281/zenodo.3583070
File: /tmp/glam_exio_tutorial/exiobase/download_log.json
History:
20241107 12:00:30 - FILEIO -  Skip download existing file IOT_2018_pxp.zip
20241107 12:00:30 - NOTE -  Download log created
20241107 12:00:30 - NOTE -  python_version: 3.9.20
20241107 12:00:30 - NOTE -  pymrio_version: 0.6.dev
20241107 12:00:30 - NOTE -  os: Linux
20241107 12:00:30 - NOTE -  hostname: NTNU09417
20241107 12:00:30 - NOTE -  username: konstans
20241107 11:56:28 - FILEIO -  Skip download existing file IOT_2018_pxp.zip
20241107 11:56:28 - NOTE -  Download log created
20241107 11:56:28 - NOTE -  python_version: 3.9.19
 ... (more lines in history)

The command downloaded the EXIOBASE 3 zip archive for given year/system if the data not already exists in the given folder.
We do not need to extract the archive, pymrio can handle all processing from the zip archive.



Next, we download the latest GLAM data. Again, the function checks if the data is already available. 

In [4]:
pymrio.GLAMprocessing.get_GLAM(storage_folder=GLAM_STORAGE_FOLDER, overwrite_existing=False)

Description: GLAM download
MRIO Name: GLAM
System: impact assessment
Version: V2024.10
File: /tmp/glam_exio_tutorial/glam/download_log.json
History:
20241107 12:00:30 - FILEIO -  Skip download existing file V1.0.2024.10.zip
20241107 12:00:30 - NOTE -  Download log created
20241107 12:00:30 - NOTE -  python_version: 3.9.20
20241107 12:00:30 - NOTE -  pymrio_version: 0.6.dev
20241107 12:00:30 - NOTE -  os: Linux
20241107 12:00:30 - NOTE -  hostname: NTNU09417
20241107 12:00:30 - NOTE -  username: konstans
20241107 11:56:32 - FILEIO -  Skip download existing file V1.0.2024.10.zip
20241107 11:56:32 - NOTE -  Download log created
20241107 11:56:32 - NOTE -  python_version: 3.9.19
 ... (more lines in history)

The download contains one single zip archive. We can keep that compressed, but we need the 
name for further processing.

In [5]:
GLAM_raw = [archive for archive in GLAM_STORAGE_FOLDER.glob("*") if archive.suffix == ".zip"][0]

Now we need to process the GLAM data:
We need to concatenate the characterization factors and the flow names into a single table, change the region classification to fit the EXIOBASE classification and rename some columns names. This can be done by calling (this takes a couple of minutes):

In [6]:
GLAM_char = pymrio.GLAMprocessing.prep_GLAM(GLAM_data=GLAM_raw)

Reading V1.0.2024.10/EQ/GLAM_template_EQ_Land_Use.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Aquatic_Microplastics.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Ecotoxicity_TR.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Freshwater_Eutrophication.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Climate_Change_FW_TR_MA.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Ecotoxicity_FW.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Marine_Eutrophication.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Terrestrial_Acidification.xlsx
Reading V1.0.2024.10/EQ/GLAM_template_EQ_Water_Consumption.xlsx
Reading V1.0.2024.10/HH/GLAM_template_HH_Climate_Change.xlsx
Reading V1.0.2024.10/HH/GLAM_template_HH_Fine_Particulate_Matter_Impacts.xlsx
Reading V1.0.2024.10/HH/GLAM_template_HH_Human_Toxicity.xlsx
Reading V1.0.2024.10/HH/GLAM_template_HH_Ionizing_Radiation.xlsx
Reading V1.0.2024.10/HH/GLAM_template_HH_Lifestyle_Impacts_Nutrition.xlsx
Reading V1.0.2024.10/HH/GLAM_template_HH_Lifestyle_Impacts_Physical_ac

This results in a long table with all characterization factors from GLAM. 
We can then later use this table to characterize EXIOBASE flows after renaming to GLAM flow names.
We can have a look at the table:

In [7]:
GLAM_char.head()

,FLOW_uuid,LCIAMethod_name__FLOW_uuid,LCIAMethod_realm__FLOW_uuid,factor,region,unit_new,unit_orig
0,NaN,EQ Land use,Terrestrial,1.318800e-15,AF,Global PDF*y/m2*y,m2*y
1,NaN,EQ Land use,Terrestrial,1.146100e-15,AF,Global PDF*y/m2*y,m2*y
2,NaN,EQ Land use,Terrestrial,1.547500e-16,AF,Global PDF*y/m2*y,m2*y
3,e1d56d4e-afe3-4b92-bd51-0a21f75e50a8,EQ Land use,Terrestrial,4.501100e-16,AF,Global PDF*y/m2*y,m2*y
4,1d443c61-ff6f-419a-99a8-9ff3055f4922,EQ Land use,Terrestrial,3.471300e-16,AF,Global PDF*y/m2*y,m2*y


We can also save the data for later use

In [8]:
GLAM_char.to_csv(GLAM_STORAGE_FOLDER / "GLAM_characterization_table.csv")

## Convert EXIOBASE stressors to GLAM flows

Here we first get the EXIOBASE GLAM bridge with:

In [9]:
exio_glam_bridge = pymrio.GLAMprocessing.get_GLAM_EXIO3_bridge()

This bride links the EXIOBASE stressors to the GLAM flow names and UUIDs.
EXIOBASE stressors are linked via [regular expressions](https://docs.python.org/3/library/re.html)
TODO: function for showing the link without regular expressions

In [10]:
exio_glam_bridge

,stressor,FLOW_name__stressor,FLOW_class0__stressor,FLOW_class1__stressor,FLOW_class2__stressor,FLOW_uuid__stressor,EXIOBASE_unit,FLOW_unit,factor,comment
0,CO2 - combustion.*,carbon dioxide (fossil),Emissions,Emissions to air,"Emissions to air, unspecified",08a91e70-3ddc-11dd-923d-0050c2490048,kg,kg,1,NaN
1,CO2 - non combustion.*,carbon dioxide (fossil),Emissions,Emissions to air,"Emissions to air, unspecified",08a91e70-3ddc-11dd-923d-0050c2490048,kg,kg,1,"Lime and Cement in EXIOBASE, not linked to gen..."
2,CO2 - waste - fossil - air,carbon dioxide (fossil),Emissions,Emissions to air,"Emissions to air, unspecified",08a91e70-3ddc-11dd-923d-0050c2490048,kg,kg,1,NaN
3,CO2 - agriculture - peat decay - air,carbon dioxide (biogenic),Emissions,Emissions to air,"Emissions to air, unspecified",08a91e70-3ddc-11dd-9c15-0050c2490048,kg,kg,1,NaN
4,CO2 - waste - biogenic - air,carbon dioxide (biogenic),Emissions,Emissions to air,"Emissions to air, unspecified",08a91e70-3ddc-11dd-9c15-0050c2490048,kg,kg,1,NaN
...,...,...,...,...,...,...,...,...,...,...
63,".*Domestic Extraction.*Limestone, gypsum, chal...",Lime,Resources,Resources from ground,Non-renewable element resources from ground,NaN,kt,kg,1000,Assumed match. Gypsum would also be there
64,.*Domestic Extraction.*Other minerals.*,lithium,Resources,Resources from ground,Non-renewable element resources from ground,fe0acd60-3ddc-11dd-aa34-0050c2490048,kt,kg,1000,Assumed match to one other mineral with high e...
65,.*Domestic Extraction.*Salt.*,Salt,Resources,Resources from ground,Non-renewable element resources from ground,NaN,kt,kg,1000,NaN
66,.*Domestic Extraction.*Slate.*,Stone (dimension),Resources,Resources from ground,Non-renewable element resources from ground,NaN,kt,kg,1000,Assumed match


We can then convert the EXIOBASE stressors to GLAM flows.
To do so, we first load the EXIOBASE 3 data we downloaded previously into memory:

In [11]:
exio3 = pymrio.parse_exiobase3(EXIOBASE_STORAGE_FOLDER / "IOT_2018_pxp.zip")

To get a clean state, we reset any pre-calculated data from the MRIO system.

In [12]:
exio3.reset_full()

We also do not need the "impact" satellite account, so we can remove that

In [13]:
del exio3.impacts

We remain with one satellite account "satellite", lets have a look:

In [14]:
print(exio3.satellite)

Extension Satellite Accounts with parameters: name, F, F_Y, S, S_Y, M, D_cba, D_pba, unit, D_cba_reg, D_pba_reg, D_imp_reg, D_exp_reg


With over 1000 stressor names:

In [15]:
exio3.satellite.F

region                                                     AT               \
sector                                             Paddy rice        Wheat   
stressor                                                                     
Taxes less subsidies on products purchased: Total           0     5.892170   
Other net taxes on production                               0   -16.782946   
Compensation of employees; wages, salaries, & e...          0     1.016108   
Compensation of employees; wages, salaries, & e...          0    13.851116   
Compensation of employees; wages, salaries, & e...          0     2.229531   
...                                                       ...          ...   
Energy Carrier Net TMAR                                     0     0.000000   
Energy Carrier Net TOTH                                     0     0.000000   
Energy Carrier Net TRAI                                     0     0.000000   
Energy Carrier Net TROA                                     0  1645.460996   
Energy Carrier Net LOSS                                     0     0.000000   

region                                                                \
sector                                             Cereal grains nec   
stressor                                                               
Taxes less subsidies on products purchased: Total          11.346024   
Other net taxes on production                             -37.361754   
Compensation of employees; wages, salaries, & e...          2.082220   
Compensation of employees; wages, salaries, & e...         28.383871   
Compensation of employees; wages, salaries, & e...          4.568782   
...                                                              ...   
Energy Carrier Net TMAR                                     0.000000   
Energy Carrier Net TOTH                                     0.000000   
Energy Carrier Net TRAI                                     0.000000   
Energy Carrier Net TROA                                   205.886745   
Energy Carrier Net LOSS                                     0.000000   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
stressor                                                                     
Taxes less subsidies on products purchased: Total                10.568132   
Other net taxes on production                                   -50.750823   
Compensation of employees; wages, salaries, & e...                3.368735   
Compensation of employees; wages, salaries, & e...               45.921051   
Compensation of employees; wages, salaries, & e...                7.391637   
...                                                                    ...   
Energy Carrier Net TMAR                                           0.000000   
Energy Carrier Net TOTH                                           0.000000   
Energy Carrier Net TRAI                                           0.000000   
Energy Carrier Net TROA                                         631.367366   
Energy Carrier Net LOSS                                           0.000000   

region                                                         \
sector                                              Oil seeds   
stressor                                                        
Taxes less subsidies on products purchased: Total    1.308766   
Other net taxes on production                       -8.062885   
Compensation of employees; wages, salaries, & e...   0.238315   
Compensation of employees; wages, salaries, & e...   3.248597   
Compensation of employees; wages, salaries, & e...   0.522907   
...                                                       ...   
Energy Carrier Net TMAR                              0.000000   
Energy Carrier Net TOTH                              0.000000   
Energy Carrier Net TRAI                              0.000000   
Energy Carrier Net TROA  

We are now ready to convert these stressors to GLAM flows. To do so we use the convert function of Pymrio.
This function can be used for many more things and is [explained in detail in the notebook here](./convert.ipynb)

In [21]:
exio3.glam_flows = exio3.satellite.convert(
    exio_glam_bridge, new_extension_name="GLAM flows",
    unit_column_orig="EXIOBASE_unit",
    unit_column_new="FLOW_unit",
    ignore_columns=["comment"]
)

This now gives us a new satellite account "glam_flows".

In [22]:
print(exio3.glam_flows)

Extension GLAM flows with parameters: name, F, F_Y, S, S_Y, M, D_cba, D_pba, unit, D_cba_reg, D_pba_reg, D_imp_reg, D_exp_reg


With flow names corresponding to GLAM flows.
Since we already had consumption based account calculated in EXIOBASE before, we can immediately see the same for the GLAM flows.

In [23]:
exio3.glam_flows.D_cba

region                                                                                                                                                               AT  \
sector                                                                                                                                                       Paddy rice   
FLOW_name                              FLOW_class0 FLOW_class1           FLOW_class2                                 FLOW_uuid                                            
Aromatic hydrocarbons, C9-17           Emissions   Emissions to air      Emissions to air, unspecified                                                     8.336762e-01   
Clay                                   Resources   Resources from ground Non-renewable element resources from ground                                       8.158067e-01   
Crude petroleum                        Resources   Resources from ground Non-renewable energy resources from ground                                        7.307133e+02   
Iron ore                               Resources   Resources from ground Non-renewable element resources from ground                                       8.225660e+00   
Lime                                   Resources   Resources from ground Non-renewable element resources from ground                                       8.475252e+00   
Nitrogen oxides                        Emissions   Emissions to air      Emissions to air, unspecified               f79d0f8f-2b0e-49cb-bed0-b1ea0fbd8625  3.478882e+03   
Phosphate rocks                        Resources   Resources from ground Non-renewable element resources from ground                                       5.066669e+00   
Phosphorus                             Emissions   Emissions to soil     Emissions to soil, unspecified              601311d7-4d5c-4d49-b131-69b73793ad0f  4.117859e+03   
                                                   Emissions to water    Emissions to water, unspecified             410e1215-9953-46ce-a878-2079353eb801  1.230884e+02   
Salt                                   Resources   Resources from ground Non-renewable element resources from ground                                       1.069828e+00   
Stone (crushed)                        Resources   Resources from ground Non-renewable element resources from ground                                       5.449720e+00   
Stone (dimension)                      Resources   Resources from ground Non-renewable element resources from ground                                       2.665247e-03   
ammonia                                Emissions   Emissions to air      Emissions to air, unspecified               08a91e70-3ddc-11dd-a2a9-0050c2490048  5.090101e+03   
arable                                 Land use    Land occupation                                                   b88d3b6d-229e-477e-bce1-e16376f75c7b  6.552293e+06   
arsenic (v)                            Emissions   Emissions to air      Emissions to air, unspecified               fd35609e-968a-4dcf-8172-c621ff680ad7  4.400803e-01   
bauxite                                Resources   Resources from ground Non-renewable element resources from ground                                       7.564544e-01   
benzo[a]pyrene                         Emissions   Emissions to air      Emissions to air, unspecified               08a91e70-3ddc-11dd-9221-0050c2490048  8.394297e-01   
benzo[b]fluoranthene                   Emissions   Emissions to air      Emissions to air, unspecified               fe0acd60-3ddc-11dd-b0de-0050c2490048  1.365720e+00   
benzo[k]fluoranthene                   Emissions   Emissions to air      Emissions to air, unspecified               4d9a8790-3ddd-11dd-94a1-0050c2490048  1.281255e-02   
cadmium (ii)                           Emissions   Emissions to air      Emissions to air, unspecified               5c69d34a-51e7-4e7d-a89c-c8536962b51a  5.906516e-02   
carbon dioxide (biogenic)              Emissions   Emissions to air

## Characterize GLAM flows

In the previous section we converted the EXIOBASE stressors to GLAM flows and prepared the GLAM characterization.
With these to things in place, we can now characterize the EXIOBASE-GLAM flows with the GLAM characterization factors.



In [25]:
# TODO: some prep in GLAM char which needs to be implmented in pymrio

# drop all nan in uuid column
# FIX: work with Flow names instead 
GLAM_char = GLAM_char.dropna(subset=["FLOW_uuid"])

# FIX: work with Flow names instead 
# replace m2*y string in unit column with m2
GLAM_char.loc[:, "unit_orig"] = GLAM_char["unit_orig"].str.replace("m2*y", "m2")

# FIX: work with Flow names instead 
# replace m2*y string in unit column with m2
GLAM_char.loc[:, "unit_orig"] = GLAM_char["unit_orig"].str.replace("kg emitted", "kg")

GLAM_char = GLAM_char.loc[GLAM_char.LCIAMethod_name__FLOW_uuid == "EQ Land use"]


In [26]:
exio3.glam_characterized = exio3.glam_flows.convert(
    GLAM_char, new_extension_name="GLAM characterized"
)

/home/konstans/proj/EXIOBASE/pymrio/pymrio/tools/ioutil.py:1148: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_orig = df_orig.stack(col)


KeyError: 'Level region not found'

This gives us a new satellite account, based on EXIOBASE stressors characterized with the newest GLAM version:

In [ ]:
print(exio3.glam_characterized)

In [ ]:
exio3.glam_characterized.D_cba